# 章 データラングリング：連結、結合、変形

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 階層型インデックス

p243
- 階層型インデックスとは、インデックスに階層を持たせることができる機能。
- 階層型インデックスを使うと、データの部分集合を簡潔に抽出できる。

__シリーズの場合__

In [2]:
# 階層型インデックス
data = pd.Series(
    np.random.randn(9),
    index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
           [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [5]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
type(data.index)

pandas.core.indexes.multi.MultiIndex

__部分インデックス参照__

In [6]:
# 親インデックスを単一指定
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [7]:
# 親インデックスの範囲指定（その１）
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [8]:
# 親インデックスの範囲指定（その２）
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

__「内側」の階層（子インデックス）を指定して抽出__

In [9]:
# 親インデックスを全て取得、子インデックスが「２」の値を抽出
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

p244
- __`unstack`__メソッドを使えば、子インデックスを列ラベルにしたデータフレームに変形することが出来る。
- __`stack`__メソッドを使えば、列ラベルを行インデックスの子インデックスにしたデータフレームに変形することが出来る。


In [15]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [16]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

__（実験）年月日の階層インデックスの場合__

In [22]:
data2 = pd.Series(["a", "b", "c", "d", "e", "f"],
                  index=[['2018', '2018', '2018', '2017', '2016', '2016'],
                         [4, 4, 4, 5, 6, 6], [1, 2, 3, 1, 1, 3]])
data2

2018  4  1    a
         2    b
         3    c
2017  5  1    d
2016  6  1    e
         3    f
dtype: object

In [23]:
data2.unstack()

,,1,2,3
2016,6,e,NaN,f
2017,5,d,NaN,NaN
2018,4,a,b,c


__データフレームの場合__

p245
- 行、列の何れの方向にも階層型インデックスを作成できる。

In [24]:
# 行と列に階層型インデックスを持たせる
frame = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

p245
- __names__属性で、階層型インデックスに名前をつけられる。（軸の一部のみにインデックス名を付ける事はできない）

In [62]:
# 階層型インデックスに名前を付ける
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

p246
- シリーズと同様に列の部分インデックス参照が出来る。
- MultiIndexオブジェクトだけを作成して、再利用することも出来る。

In [28]:
# 部分インデックス参照
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [60]:
# 階層インデックスの手動作成
pd.MultiIndex.from_arrays(
    [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
    names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 階層の順序変更やソート

p246
- インデックス階層の順序を入れ替えるには、__`swaplevel`__メソッドを使う。
- 特定のインデックスでソートしたい場合は__`sort_inde`__メソッドを使う。

In [65]:
# 入れ替え前
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [66]:
# key1 と key2 を入れ替える。
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [67]:
# key2に対してソート（key1はlevel=0）
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [68]:
# key1とkey2を入れ替えた上で、一番外の行インデックスでソート
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 階層ごとの要約統計量

p246
- levelオプションを使うと、その軸での集計対象としたい階層を指定できる。（デフォルトは行:axis=0）

In [69]:
# key2に対する合計
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [80]:
# color列に対する合計
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

？__key1、key2もまとめて集計してみるには？__

### データフレームの列をインデックスに使う

p248
- __`set_index`__メソッドは、指定した1つ以上の列をインデックスとするデータフレームを新規作成する。
- __`drop=False`__で、インデックスに使用した列を残したまま、データフレームを作成できる。
- __reset_index__メソッドで__`set_index`__で作成したインデックスを列に戻す。

In [82]:
frame = pd.DataFrame({
    'a': range(7),
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0, 1, 2, 0, 1, 2, 3]
})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [84]:
# c、d列をインデックスとするデータフレームを作成
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [85]:
# インデックスに使用した列を残す。
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [86]:
# 行インデックスを列インデックスに戻す
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## データセットの結合とマージ

p249
- __`pandas.merge`__でデータフレームの行同士をキーで結合する。
- __`pandas.concat`__で縦や横方向に結合する。
- __`combine_first`__で重複するデータを持つ複数のオブジェクトをつなぎ合わせて、オブジェクトの欠損値を別のオブジェクトの値で穴埋めできる。

### データフレームをデータベース風に結合する
p249
- __`pandas.merge`__でデータフレームの行同士をキーで結合する。
- merge関数の引数は表8-2 を参照。(p254）

In [91]:
df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': range(7)
})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [89]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


p251
- デフォルトとして、どちらのデータフレームにも含まれる同じ列ラベルをキーとして用いる。

In [94]:
# df1とf2を列ラベルで内部結合する（片側にしか存在しない列ラベルの値は結合されない）
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


p251
- 結合キーは__`on`__で指定可能（推奨）。

In [93]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


p251
- キーとしたい列の名前が2つのデータフレームで異なっている場合、__`left_on`__、__`right_on`__ で、それぞれを個別に指定する。

In [95]:
df3 = pd.DataFrame({
    'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': range(7)
})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [97]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [98]:
# データフレーム毎に結合キーを指定
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


p251
- 結合の方法は__`how`__で指定できる（デフォルトはinner）。

In [99]:
# df1とdf2を外部結合（両データフレームのキーの和集合）
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


__例）左 外部結合__  
__`merge`__の第１引数のデータフレームに存在するキーで結合される。

In [110]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)}).sort_values(by="key") 
df1

,key,data1
2,a,2
4,a,4
0,b,0
1,b,1
5,b,5
3,c,3


In [111]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)}).sort_values(by="key") 
df2

,key,data2
0,a,0
2,a,2
1,b,1
3,b,3
4,d,4


In [112]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,a,2,0.0
1,a,2,2.0
2,a,4,0.0
3,a,4,2.0
4,b,0,1.0
5,b,0,3.0
6,b,1,1.0
7,b,1,3.0
8,b,5,1.0
9,b,5,3.0


__例）内部結合__  
両データフレームに存在するキーで結合される。

In [114]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,a,2,0
1,a,2,2
2,a,4,0
3,a,4,2
4,b,0,1
5,b,0,3
6,b,1,1
7,b,1,3
8,b,5,1
9,b,5,3


__例）外部結合__  
少なくとも何れかのデータフレームに存在するキーで結合される。

In [115]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)}).sort_values(by="key") 
df1

,key,data1
2,a,2
4,a,4
0,b,0
1,b,1
5,b,5
3,c,3


In [116]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)}).sort_values(by="key") 
df2

,key,data2
0,a,0
2,a,2
1,b,1
3,b,3
4,d,4


In [118]:
pd.merge(df1, df2, on="key", how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,2.0,2.0
2,a,4.0,0.0
3,a,4.0,2.0
4,b,0.0,1.0
5,b,0.0,3.0
6,b,1.0,1.0
7,b,1.0,3.0
8,b,5.0,1.0
9,b,5.0,3.0


p253
- 複数のキーでマージを行いたい場合は、__`on`__に列名のリストを渡す。

In [124]:
left = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]
}).sort_values(by=["key1", "key2"])
left

,key1,key2,lval
2,bar,one,3
0,foo,one,1
1,foo,two,2


In [125]:
right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'two'],
    'rval': [4, 5, 6, 7]
}).sort_values(by=["key1", "key2"])
right

,key1,key2,rval
2,bar,one,6
3,bar,two,7
0,foo,one,4
1,foo,one,5


In [126]:
# key1、key2で結合
pd.merge(left, right, on=['key1', 'key2'])

,key1,key2,lval,rval
0,bar,one,3,6
1,foo,one,1,4
2,foo,one,1,5


p254
- __`suffixes`__オプションで、列名が重複した場合に、左のデータフレーム由来の列と右のデータフレーム由来の列のそれぞれの名前の末尾に加える文字列を指定できる。

In [129]:
# 列名key2 が重複
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,bar,one,3,one,6
1,bar,one,3,two,7
2,foo,one,1,one,4
3,foo,one,1,one,5
4,foo,two,2,one,4
5,foo,two,2,one,5


### インデックスによるマージ

p255
- 列ではなく行ラベルでマージしたい場合、__`left_index=True`__ または __`right_index=True`__（若しくは両オ
プション）を使用する。

In [141]:
left1 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'data1': range(6)
})
left1

,key,data1
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [140]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


__right1__の行インデックスをキーに__left1__の__key__列と結合する。

In [142]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,data1,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth
righth

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2
right2
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

In [ ]:
left2.join(right2, how='outer')

In [ ]:
left1.join(right1, on='key')

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another
left2.join([right2, another])
left2.join([right2, another], how='outer')

### Concatenating Along an Axis

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
pd.concat([s1, s2, s3])

In [ ]:
pd.concat([s1, s2, s3], axis=1)

In [ ]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis=1)
pd.concat([s1, s4], axis=1, join='inner')

In [ ]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
result.unstack()

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1
df2
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

In [ ]:
pd.concat([df1, df2], ignore_index=True)

### Combining Data with Overlap

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a
b
np.where(pd.isnull(a), b, a)

In [ ]:
b[:-2].combine_first(a[2:])

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack(0)
result.unstack('state')

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

In [ ]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

### Pivoting “Long” to “Wide” Format

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### Pivoting “Wide” to “Long” Format

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

## Conclusion